In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.datasets import fetch_mldata
from sklearn import metrics
# Ignore warnings
import warnings
warnings.filterwarnings('ignore');

In [2]:
# US Income data set path
income_path = "Resources/income.csv"

# Import the data into a Pandas DataFrame
df = pd.read_csv(income_path)
df.head()

39          State-gov   77516   Bachelors   13        Never-married  \
0  50   Self-emp-not-inc   83311   Bachelors   13   Married-civ-spouse   
1  38            Private  215646     HS-grad    9             Divorced   
2  53            Private  234721        11th    7   Married-civ-spouse   
3  28            Private  338409   Bachelors   13   Married-civ-spouse   
4  37            Private  284582     Masters   14   Married-civ-spouse   

         Adm-clerical   Not-in-family   White     Male   2174   0   40  \
0     Exec-managerial         Husband   White     Male      0   0   13   
1   Handlers-cleaners   Not-in-family   White     Male      0   0   40   
2   Handlers-cleaners         Husband   Black     Male      0   0   40   
3      Prof-specialty            Wife   Black   Female      0   0   40   
4     Exec-managerial            Wife   White   Female      0   0   40   

    United-States   <=50K  
0   United-States   <=50K  
1   United-States   <=50K  
2   United-States   <=50K  
3            Cuba   <=50K  
4   United-States   <=50K

In [3]:
columns = ['Age',
           'Employer_Type',
           'fnlgwt',
           'Education',
           'Ed_Numeric',
           'Marital_Status',
           'Occupation',
           'Relationship',
           'Race',
           'Sex',
           'Capital_Gain',
           'Capital_Loss',
           'Hours_per_Week',
           'Country',
           'Income_Above_Below_50K']
df=pd.read_csv(income_path, names=columns)
df.head(10)

Age      Employer_Type  fnlgwt   Education  Ed_Numeric  \
0   39          State-gov   77516   Bachelors          13   
1   50   Self-emp-not-inc   83311   Bachelors          13   
2   38            Private  215646     HS-grad           9   
3   53            Private  234721        11th           7   
4   28            Private  338409   Bachelors          13   
5   37            Private  284582     Masters          14   
6   49            Private  160187         9th           5   
7   52   Self-emp-not-inc  209642     HS-grad           9   
8   31            Private   45781     Masters          14   
9   42            Private  159449   Bachelors          13   

           Marital_Status          Occupation    Relationship    Race  \
0           Never-married        Adm-clerical   Not-in-family   White   
1      Married-civ-spouse     Exec-managerial         Husband   White   
2                Divorced   Handlers-cleaners   Not-in-family   White   
3      Married-civ-spouse   Handlers-cleaners         Husband   Black   
4      Married-civ-spouse      Prof-specialty            Wife   Black   
5      Married-civ-spouse     Exec-managerial            Wife   White   
6   Married-spouse-absent       Other-service   Not-in-family   Black   
7      Married-civ-spouse     Exec-managerial         Husband   White   
8           Never-married      Prof-specialty   Not-in-family   White   
9      Married-civ-spouse     Exec-managerial         Husband   White   

       Sex  Capital_Gain  Capital_Loss  Hours_per_Week         Country  \
0     Male          2174             0              40   United-States   
1     Male             0             0              13   United-States   
2     Male             0             0              40   United-States   
3     Male             0             0              40   United-States   
4   Female             0             0              40            Cuba   
5   Female             0             0              40   United-States   
6   Female             0             0              16         Jamaica   
7     Male             0             0              45   United-States   
8   Female         14084             0              50   United-States   
9     Male          5178             0              40   United-States   

  Income_Above_Below_50K  
0                  <=50K  
1                  <=50K  
2                  <=50K  
3                  <=50K  
4                  <=50K  
5                  <=50K  
6                  <=50K  
7                   >50K  
8                   >50K  
9                   >50K

In [4]:
# Check for null values
print ("\nNull Values :  ", df.isnull().sum().values.sum())


Null Values :   0


In [5]:
# Check for " ?" values
df_missing = (df==' ?').sum()
df_missing

Age                          0
Employer_Type             1836
fnlgwt                       0
Education                    0
Ed_Numeric                   0
Marital_Status               0
Occupation                1843
Relationship                 0
Race                         0
Sex                          0
Capital_Gain                 0
Capital_Loss                 0
Hours_per_Week               0
Country                    583
Income_Above_Below_50K       0
dtype: int64

In [6]:
# Remove rows which have invalid data from data frame
df = df[(df.astype(str) != ' ?').all(axis=1)]

# Make sure that the rows were dropped. There were 32561 rows before.
len(df)
df.shape

(30162, 15)

In [7]:
# Create a new column for income which is binary
df['Income_Above/Below_50K'] = df.apply(lambda row: 
                                        1 if '>50K' in row['Income_Above_Below_50K'] 
                                        else 0, axis=1)

In [8]:
# Remove unwanted columns
df = df.drop(['Income_Above_Below_50K'], axis=1)
df.head(10)

Age      Employer_Type  fnlgwt   Education  Ed_Numeric  \
0   39          State-gov   77516   Bachelors          13   
1   50   Self-emp-not-inc   83311   Bachelors          13   
2   38            Private  215646     HS-grad           9   
3   53            Private  234721        11th           7   
4   28            Private  338409   Bachelors          13   
5   37            Private  284582     Masters          14   
6   49            Private  160187         9th           5   
7   52   Self-emp-not-inc  209642     HS-grad           9   
8   31            Private   45781     Masters          14   
9   42            Private  159449   Bachelors          13   

           Marital_Status          Occupation    Relationship    Race  \
0           Never-married        Adm-clerical   Not-in-family   White   
1      Married-civ-spouse     Exec-managerial         Husband   White   
2                Divorced   Handlers-cleaners   Not-in-family   White   
3      Married-civ-spouse   Handlers-cleaners         Husband   Black   
4      Married-civ-spouse      Prof-specialty            Wife   Black   
5      Married-civ-spouse     Exec-managerial            Wife   White   
6   Married-spouse-absent       Other-service   Not-in-family   Black   
7      Married-civ-spouse     Exec-managerial         Husband   White   
8           Never-married      Prof-specialty   Not-in-family   White   
9      Married-civ-spouse     Exec-managerial         Husband   White   

       Sex  Capital_Gain  Capital_Loss  Hours_per_Week         Country  \
0     Male          2174             0              40   United-States   
1     Male             0             0              13   United-States   
2     Male             0             0              40   United-States   
3     Male             0             0              40   United-States   
4   Female             0             0              40            Cuba   
5   Female             0             0              40   United-States   
6   Female             0             0              16         Jamaica   
7     Male             0             0              45   United-States   
8   Female         14084             0              50   United-States   
9     Male          5178             0              40   United-States   

   Income_Above/Below_50K  
0                       0  
1                       0  
2                       0  
3                       0  
4                       0  
5                       0  
6                       0  
7                       1  
8                       1  
9                       1

# Encoding Categorical Variables

In [9]:
from sklearn import preprocessing

# select all categorical variables
df_categorical = df.select_dtypes(include=['object'])
df_categorical.head()

Employer_Type   Education       Marital_Status          Occupation  \
0          State-gov   Bachelors        Never-married        Adm-clerical   
1   Self-emp-not-inc   Bachelors   Married-civ-spouse     Exec-managerial   
2            Private     HS-grad             Divorced   Handlers-cleaners   
3            Private        11th   Married-civ-spouse   Handlers-cleaners   
4            Private   Bachelors   Married-civ-spouse      Prof-specialty   

     Relationship    Race      Sex         Country  
0   Not-in-family   White     Male   United-States  
1         Husband   White     Male   United-States  
2   Not-in-family   White     Male   United-States  
3         Husband   Black     Male   United-States  
4            Wife   Black   Female            Cuba

In [10]:

# encode categorical variables using label Encoder
# apply label encoder to df_categorical

le = preprocessing.LabelEncoder()
df_categorical = df_categorical.apply(le.fit_transform)
df_categorical.head()

Employer_Type  Education  Marital_Status  Occupation  Relationship  Race  \
0              5          9               4           0             1     4   
1              4          9               2           3             0     4   
2              2         11               0           5             1     4   
3              2          1               2           5             0     2   
4              2          9               2           9             5     2   

   Sex  Country  
0    1       38  
1    1       38  
2    1       38  
3    1       38  
4    0        4

In [11]:
# Next, Concatenate df_categorical dataframe with original df (dataframe)

# first, Drop earlier duplicate columns which had categorical values
df = df.drop(df_categorical.columns,axis=1)
df = pd.concat([df,df_categorical],axis=1)
df.head()

Age  fnlgwt  Ed_Numeric  Capital_Gain  Capital_Loss  Hours_per_Week  \
0   39   77516          13          2174             0              40   
1   50   83311          13             0             0              13   
2   38  215646           9             0             0              40   
3   53  234721           7             0             0              40   
4   28  338409          13             0             0              40   

   Income_Above/Below_50K  Employer_Type  Education  Marital_Status  \
0                       0              5          9               4   
1                       0              4          9               2   
2                       0              2         11               0   
3                       0              2          1               2   
4                       0              2          9               2   

   Occupation  Relationship  Race  Sex  Country  
0           0             1     4    1       38  
1           3             0     4    1       38  
2           5             1     4    1       38  
3           5             0     2    1       38  
4           9             5     2    0        4

In [12]:
# look at column type
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 30162 entries, 0 to 32560
Data columns (total 15 columns):
Age                       30162 non-null int64
fnlgwt                    30162 non-null int64
Ed_Numeric                30162 non-null int64
Capital_Gain              30162 non-null int64
Capital_Loss              30162 non-null int64
Hours_per_Week            30162 non-null int64
Income_Above/Below_50K    30162 non-null int64
Employer_Type             30162 non-null int32
Education                 30162 non-null int32
Marital_Status            30162 non-null int32
Occupation                30162 non-null int32
Relationship              30162 non-null int32
Race                      30162 non-null int32
Sex                       30162 non-null int32
Country                   30162 non-null int32
dtypes: int32(8), int64(7)
memory usage: 2.8 MB


# Train and Test Data

In [13]:
#from sklearn.Cross_validation import train_test_split

X = df.drop(['Income_Above/Below_50K'], axis = 1)
y = df['Income_Above/Below_50K']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)

# PCA to Speed-up Machine Learning Algorithms

In [14]:
from sklearn import preprocessing

categorical = ['Employer_Type', 'Education', 'Marital_Status', 'Occupation', 'Relationship', 'Race', 'Sex']
for feature in categorical:
        le = preprocessing.LabelEncoder()
        X_train[feature] = le.fit_transform(X_train[feature])
        X_test[feature] = le.transform(X_test[feature])

In [15]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

X_train = pd.DataFrame(scaler.fit_transform(X_train), columns = X.columns)

X_test = pd.DataFrame(scaler.transform(X_test), columns = X.columns)

In [16]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

logreg = LogisticRegression()
logreg.fit(X_train, y_train)
y_pred = logreg.predict(X_test)

print('Logistic Regression accuracy score with all the features: {0:0.4f}'. format(accuracy_score(y_test, y_pred)))

Logistic Regression accuracy score with all the features: 0.8133


In [17]:
from sklearn.decomposition import PCA
pca = PCA()
X_train = pca.fit_transform(X_train)
pca.explained_variance_ratio_

array([0.14860046, 0.10150752, 0.08299841, 0.07943599, 0.07390192,
       0.07256705, 0.06955647, 0.06759116, 0.06418322, 0.06022321,
       0.05983379, 0.0487621 , 0.04333408, 0.02750461])

In [19]:
X = df.drop(['Income_Above/Below_50K','Country'], axis=1)
y = df['Income_Above/Below_50K']


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)


categorical = ['Employer_Type', 'Education', 'Marital_Status', 'Occupation', 'Relationship', 'Race', 'Sex']
for feature in categorical:
        le = preprocessing.LabelEncoder()
        X_train[feature] = le.fit_transform(X_train[feature])
        X_test[feature] = le.transform(X_test[feature])


X_train = pd.DataFrame(scaler.fit_transform(X_train), columns = X.columns)

X_test = pd.DataFrame(scaler.transform(X_test), columns = X.columns)

logreg = LogisticRegression()
logreg.fit(X_train, y_train)
y_pred = logreg.predict(X_test)

print('Logistic Regression accuracy score with the first 13 features: {0:0.4f}'. format(accuracy_score(y_test, y_pred)))

Logistic Regression accuracy score with the first 13 features: 0.8133


In [22]:
X = df.drop(['Income_Above/Below_50K','Country', 'Hours_per_Week'], axis=1)
y = df['Income_Above/Below_50K']


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)


categorical = ['Employer_Type', 'Education', 'Marital_Status', 'Occupation', 'Relationship', 'Race', 'Sex']
for feature in categorical:
        le = preprocessing.LabelEncoder()
        X_train[feature] = le.fit_transform(X_train[feature])
        X_test[feature] = le.transform(X_test[feature])


X_train = pd.DataFrame(scaler.fit_transform(X_train), columns = X.columns)

X_test = pd.DataFrame(scaler.transform(X_test), columns = X.columns)

logreg = LogisticRegression()
logreg.fit(X_train, y_train)
y_pred = logreg.predict(X_test)

print('Logistic Regression accuracy score with the first 12 features: {0:0.4f}'. format(accuracy_score(y_test, y_pred)))

Logistic Regression accuracy score with the first 12 features: 0.8122


In [23]:
X = df.drop(['Income_Above/Below_50K','Country', 'Hours_per_Week', 'Capital_Loss'], axis=1)
y = df['Income_Above/Below_50K']


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)


categorical = ['Employer_Type', 'Education', 'Marital_Status', 'Occupation', 'Relationship', 'Race', 'Sex']
for feature in categorical:
        le = preprocessing.LabelEncoder()
        X_train[feature] = le.fit_transform(X_train[feature])
        X_test[feature] = le.transform(X_test[feature])


X_train = pd.DataFrame(scaler.fit_transform(X_train), columns = X.columns)

X_test = pd.DataFrame(scaler.transform(X_test), columns = X.columns)

logreg = LogisticRegression()
logreg.fit(X_train, y_train)
y_pred = logreg.predict(X_test)

print('Logistic Regression accuracy score with the first 11 features: {0:0.4f}'. format(accuracy_score(y_test, y_pred)))

Logistic Regression accuracy score with the first 11 features: 0.8115


In [24]:
X = df.drop(['Income_Above/Below_50K'], axis=1)
y = df['Income_Above/Below_50K']


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)


categorical = ['Employer_Type', 'Education', 'Marital_Status', 'Occupation', 'Relationship', 'Race', 'Sex', 'Country']
for feature in categorical:
        le = preprocessing.LabelEncoder()
        X_train[feature] = le.fit_transform(X_train[feature])
        X_test[feature] = le.transform(X_test[feature])


X_train = pd.DataFrame(scaler.fit_transform(X_train), columns = X.columns)


pca= PCA()
pca.fit(X_train)
cumsum = np.cumsum(pca.explained_variance_ratio_)
dim = np.argmax(cumsum >= 0.90) + 1
print('The number of dimensions required to preserve 90% of variance is',dim)

The number of dimensions required to preserve 90% of variance is 12


In [36]:
X = df.drop(['Income_Above/Below_50K'], axis=1)
y = df['Income_Above/Below_50K']


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)


categorical = [ 'Employer_Type', 'Education', 'Marital_Status', 'Occupation', 'Relationship', 'Race', 'Sex',]
for feature in categorical:
        le = preprocessing.LabelEncoder()
        X_train[feature] = le.fit_transform(X_train[feature])
        X_test[feature] = le.transform(X_test[feature])


X_train = pd.DataFrame(scaler.fit_transform(X_train), columns = X.columns)

X_test = pd.DataFrame(scaler.transform(X_test), columns = X.columns)

logreg = LogisticRegression()
logreg.fit(X_train, y_train)
y_pred = logreg.predict(X_test)

print('Logistic Regression accuracy score with the first 11 features: {0:0.4f}'. format(accuracy_score(y_test, y_pred)))

Logistic Regression accuracy score with the first 11 features: 0.8133
